In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .master('local[8]')
    .config('spark.driver.memory', '16g')
    .getOrCreate()
)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/09 19:31:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/06/09 19:31:27 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/06/09 19:31:27 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
test_data = spark.read.parquet('../data/amex-default-prediction/test_data')
train_data = spark.read.parquet('../data/amex-default-prediction/train_data')
train_labels = spark.read.parquet('../data/amex-default-prediction/train_labels')

In [3]:
test_data.groupBy('customer_ID').count().orderBy('count', ascending=False).describe().show()

+-------+--------------------+-----------------+
|summary|         customer_ID|            count|
+-------+--------------------+-----------------+
|  count|              924621|           924621|
|   mean|                null|12.29018376177915|
| stddev|                null|2.253139051478346|
|    min|00000469ba478561f...|                1|
|    max|fffffa7cf7e453e1a...|               13|
+-------+--------------------+-----------------+



The train and test counts look similar..

In [4]:
train_data.groupBy('customer_ID').count().orderBy('count', ascending=False).describe().show()

+-------+--------------------+------------------+
|summary|         customer_ID|             count|
+-------+--------------------+------------------+
|  count|              458913|            458913|
|   mean|                null|12.053376130116165|
| stddev|                null|2.6102734612943537|
|    min|0000099d6bd597052...|                 1|
|    max|fffff1d38b785cef8...|                13|
+-------+--------------------+------------------+



Oh we're expected to predict for an entirely new set of customers..

In [5]:
test_ids = test_data.select('customer_ID').distinct()
train_data.join(test_ids, on='customer_ID', how='inner').groupBy('customer_ID').count().orderBy('count', ascending=False).describe().show()

+-------+-----------+-----+
|summary|customer_ID|count|
+-------+-----------+-----+
|  count|          0|    0|
|   mean|       null| null|
| stddev|       null| null|
|    min|       null| null|
|    max|       null| null|
+-------+-----------+-----+

